In [2]:
#Numpy for array related functions.
import numpy as np
#Dlib for deep learning based Modules and face landmark detection.
import dlib
#Import OpenCV Library for basic image processing functions.
import cv2
#Import threading to run multiple threads(tasks, function calls)at the same time.
import threading
from threading import Thread
#face_utils for basic operations of conversion
import imutils
from imutils import face_utils
#scipy to solve scientific and mathematical problems
from scipy.spatial import distance as dist
#pygame for playing alarm
import pygame

def sound_alarm():
    pygame.mixer.init()
    pygame.mixer.music.load("alarm.wav")
    pygame.mixer.music.play()
    
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])   #vertical distance
    C = dist.euclidean(eye[0], eye[3])   #horizontal distance

    ear = (A + B) / (2.0 * C)
    return ear

EYE_AR_THRESH = float (input("Enter your approx open eye EAR ratio: "))
EYE_AR_CONSEC_FRAMES = 50


COUNTER = 0
ALARM_ON = False

#Grab the indexes of the facial landmarks for the left and
# right eye,respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
( rStart , rEnd ) =face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#this will capture video from your webcam
cap = cv2.VideoCapture(1)

#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Failed to capture frame from camera. Check camera index in cv2.VideoCapture(0) \n')
        break
        cv2.imshow(frame)

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray)

    for rect in rects:
        x1 = rect.left()
        y1 = rect.top()
        x2 = rect.right()
        y2 = rect.bottom()
        
        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        landmarks = predictor(gray, rect)
        landmarks = face_utils.shape_to_np(landmarks)
        
        shape = predictor(gray, rect)    # determine the facial landmarks for face region
        shape = face_utils.shape_to_np(shape) #converting to numpy array

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not ALARM_ON:
                    ALARM_ON = True
                    d=threading.Thread(target=sound_alarm)
                    d.setDaemon(True)
                    d.start()

                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            ALARM_ON = False

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        for n in range(0, 68):
            (x,y) = landmarks[n]
            
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    cv2.imshow("Frame", frame)
    #cv2.imshow("Face landmarks detection", face_frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):#by pressing 'q' you can stop the program 
        break

cv2.destroyAllWindows()
cap.release()

Enter your approx open eye EAR ratio: .22
